---
## **INTRODUÇÃO**

---
## **DESENVOLVIMENTO**

---
### **DOWNLOAD DO DATASET**

Para iniciar nossa análise, utilizaremos o dataset "Student Performance Factors" disponível no Kaggle. Realizamos o download do conjunto de dados através do link  https://teams.microsoft.com/l/message/19:e98de34e8e034b56b694d6cdac057368@thread.v2/1760965275974?context=%7B%22contextType%22%3A%22chat%22%7D e extraímos o arquivo compactado em formato ZIP. O arquivo resultante, em formato CSV, foi armazenado no diretório da Quest 4 do projeto no Jupyter Notebook. Agora, procederemos com a leitura e carregamento dos dados para dar início ao processo de exploração e modelagem

In [1]:
import pandas as pd

df = pd.read_csv('StudentPerformanceFactors.csv')
print(df)

      Hours_Studied  Attendance Parental_Involvement Access_to_Resources  \
0                23          84                  Low                High   
1                19          64                  Low              Medium   
2                24          98               Medium              Medium   
3                29          89                  Low              Medium   
4                19          92               Medium              Medium   
...             ...         ...                  ...                 ...   
6602             25          69                 High              Medium   
6603             23          76                 High              Medium   
6604             20          90               Medium                 Low   
6605             10          86                 High                High   
6606             15          67               Medium                 Low   

     Extracurricular_Activities  Sleep_Hours  Previous_Scores  \
0                     

---
### **TRATAMENTO DE DADOS**

No tratamento de dados, será realizado três passos:
1. Retirar linhas que não tenham algum valor, para evitar erros no treinamento;
2. Converter os dados categóricos (ordinais e binários) para numéricos, já que nosso target é numérico e usaremos regressão para o treinamento;
3. Dividir os dados entre treino e teste para aplicar no modelo.
   
Vamos começar com o primeiro tópico:

In [2]:
import seaborn as sns #para carregar datasets prontos;
import pandas #para manipulação de dados;
from sklearn.preprocessing import OrdinalEncoder, StandardScaler #para conversão de dados categóricos;

df = df.dropna()

# Drop de colunas com QUALQUER linha NaN
df = df.dropna(axis=1)

#### **CONVERSÃO DE DADOS CATEGÓRICOS PARA NUMÉRICOS**

Abaixo segue o código que irá converter dados categóricos ordinais com o OrdinalEncoder, atribuindo valores numéricos correspondentes à ordem de cada categoria, e dados binários a partir de "mapping" cada label com 0 ou 1.

In [3]:
categoricos = {
    'Parental_Involvement': ['Low', 'Medium', 'High'],
    'Access_to_Resources': ['Low', 'Medium', 'High'],
    'Motivation_Level': ['Low', 'Medium', 'High'],
    'Family_Income': ['Low', 'Medium', 'High'],
    'Teacher_Quality': ['Low', 'Medium', 'High'],
    'Distance_from_Home': ['Near', 'Moderate', 'Far'],
    'Peer_Influence': ['Negative', 'Neutral', 'Positive'],
    'Parental_Education_Level': ['High School', 'College', 'Postgraduate'] 
}

binarios = {
    'Gender': {'Male': 0, 'Female': 1},
    'Learning_Disabilities': {'No': 0, 'Yes': 1},
    'Extracurricular_Activities': {'No': 0, 'Yes': 1},
    'Internet_Access': {'No': 0, 'Yes': 1},
    'School_Type': {'Public': 0, 'Private': 1}
}

for coluna, ordem in categoricos.items():
    encoder = OrdinalEncoder(categories=[ordem])
    df[coluna] = encoder.fit_transform(df[[coluna]])

for coluna, mapa in binarios.items():
    df[coluna] = df[coluna].map(mapa)


#### **SPLIT E NORMALIZAÇÃO**

Nesta seção, iremos dividir nossa dataset entre treino e teste para poder criar e treinar os modelos. Além disso, vamos definir nossos atributos e o nosso target, que será o exam score. Com esses objetivos, vamos usar funções do `sklearn.model_selection`. Para a normalização, foi escolhido o `StandardScaler`.

Note que a normalização com StandardScaler é crucial para KNN, Ridge Regression e Elastic Net devido à sua sensibilidade à escala dos dados. O KNN baseia-se em cálculos de distância onde features em escalas diferentes distorceriam as medidas de similaridade. Já Ridge e Elastic Net aplicam penalidades de regularização que seriam injustamente influenciadas por variáveis com magnitude naturalmente maior, comprometendo a eficácia do modelo. O StandardScaler padroniza os dados removendo a média e escalando para variância unitária, transformando cada feature para ter média zero e desvio padrão igual a 1. Essa transformação garante que todas as features contribuam igualmente para os algoritmos, independentemente de suas escalas originais.

In [4]:
print(df.columns)

Index(['Hours_Studied', 'Attendance', 'Parental_Involvement',
       'Access_to_Resources', 'Extracurricular_Activities', 'Sleep_Hours',
       'Previous_Scores', 'Motivation_Level', 'Internet_Access',
       'Tutoring_Sessions', 'Family_Income', 'Teacher_Quality', 'School_Type',
       'Peer_Influence', 'Physical_Activity', 'Learning_Disabilities',
       'Parental_Education_Level', 'Distance_from_Home', 'Gender',
       'Exam_Score'],
      dtype='object')


In [5]:
from sklearn.model_selection import train_test_split #Para dividir entre treino e teste do modelo

FEATURES = ['Hours_Studied', 'Attendance', 'Parental_Involvement',
       'Access_to_Resources', 'Extracurricular_Activities', 'Sleep_Hours',
       'Previous_Scores', 'Motivation_Level', 'Internet_Access',
       'Tutoring_Sessions', 'Family_Income', 'Teacher_Quality', 'School_Type',
       'Peer_Influence', 'Physical_Activity', 'Learning_Disabilities',
       'Parental_Education_Level', 'Distance_from_Home', 'Gender']
#definindo o alvo que será previsto
TARGET = ['Exam_Score']
    
#definindo na variável X os valores de cada coluna que é parâmetro
X = df[FEATURES].values
    
#definindo na variável y os valores da coluna alvo e transformando em unidimensional com o método ".ravel()"
y = df[TARGET].values.ravel()

#dividindo entre teste e treino, com uma porcentagem de 20% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

normalizador = StandardScaler()
X_treino_norm = normalizador.fit_transform(X_treino)
X_teste_norm = normalizador.transform(X_teste)

---
### **BASELINE**

Nesta seção, estabeleceremos uma baseline de performance como referência fundamental para avaliar a eficácia dos nossos modelos de regressão. A baseline representa o desempenho mínimo que nossos algoritmos devem superar para demonstrar valor preditivo. Utilizaremos abordagens simples, prevendo constantemente a média dos valores de treino. Esta prática é crucial para validar que a complexidade adicional dos algoritmos de machine learning está de fato gerando ganhos preditivos significativos em relação a soluções triviais.

In [6]:
from sklearn.metrics import root_mean_squared_error
import numpy as np

# Calcula a média do target no treino
media = np.mean(y_treino)

# Cria previsões (sempre a média)
y_pred_baseline = np.full_like(y_teste, fill_value=media)

# Calcula RMSE
RMSE = root_mean_squared_error(y_teste, y_pred_baseline)

print(RMSE)

3.9513102465608547


---
### **CRIANDO O MODELO E TREINANDO**

Nesta seção, aplicaremos cinco algoritmos de regressão distintos - K-Nearest Neighbors (KNN), Random Forest, Elastic Net, Gradient Boosting Regressor e Ridge Regression - para treinar modelos preditivos utilizando nosso conjunto de dados. Para cada algoritmo, iniciaremos com uma fundamentação teórica que explica seus princípios de funcionamento e características principais, seguida pela implementação prática utilizando a biblioteca scikit-learn.

É importante destacar que, nesta fase inicial de modelagem, utilizaremos os valores padrão de hiperparâmetros definidos pelo scikit-learn para cada algoritmo. Esta abordagem nos permitirá estabelecer uma linha de base de performance antes de proceder com a otimização sistemática de hiperparâmetros, que será realizada em etapas subsequentes do projeto.

Para avalisar o desempenho dos algoritmos neste primeiro momento, será usado o Root Mean Squared Error (RMSE) - uma métrica que quantifica a magnitude média dos erros de previsão. É a raiz quadrada da média dos quadrados das diferenças entre valores observados e previstos e está na mesma unidade da variável target. Por utilizar o quadrado dos erros, esta métrica penaliza mais fortemente previsões que estão significativamente distantes dos valores reais, sendo particularmente sensível a outliers.

#### **K-nearest neighbors (KNN)**

O algoritmo K-Nearest Neighbors (KNN) é um método de aprendizado baseado em instâncias que realiza previsões calculando a similaridade entre observações no espaço de features. Quando aplicado a problemas de regressão, o KNN identifica os k pontos de dados mais próximos da observação que se deseja prever, utilizando métricas de distância como Euclidiana ou Manhattan, e então calcula a média dos valores target desses vizinhos para gerar a previsão final. Esta abordagem não constrói um modelo explícito durante o treinamento, mas armazena todo o conjunto de dados, realizando os cálculos apenas no momento da inferência, o que o torna computacionalmente intensivo para conjuntos 
grandes, porém flexível para capturar padrões complexos não-lineares nos dados.

In [7]:
from sklearn.neighbors import KNeighborsRegressor #Para aplicar o modelo
from sklearn.metrics import root_mean_squared_error #Para avaliar seu desempenho
 

scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

#criando modelo
modelo_knn = KNeighborsRegressor()
modelo_knn.fit(X_treino_scaled, y_treino)
    
#treinando o modelo
y_pred_knn = modelo_knn.predict(X_teste_scaled)

#avaliando o desempenho com RMSE (que será nosso retorno) a partir do target previsto
RMSE = root_mean_squared_error(y_teste, y_pred_knn)

print(RMSE)

2.783189573246307


#### **Random Forest**

Random Forest Regression é um método de ensemble baseado em bagging (Bootstrap Aggregating) que constrói múltiplas árvores de decisão independentes e combina suas previsões através da média. O bagging é uma técnica que reduz a variância do modelo através da criação de múltiplos conjuntos de treinamento via amostragem com reposição (bootstrap) e da agregação de seus resultados. Durante o treinamento, cada árvore é construída usando uma amostra bootstrap do conjunto de dados original e, em cada divisão de nó, apenas um subconjunto aleatório de features é considerado para seleção, introduzindo dupla aleatorização que promove diversidade entre as árvores e reduz significativamente o overfitting, resultando em um modelo robusto que geralmente apresenta excelente performance com pouca necessidade de ajuste fino de hiperparâmetros e natural resistência a outliers e ruídos nos dados.

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

# Criando modelo
modelo_rf = RandomForestRegressor()
    
# Treinando o modelo
modelo_rf.fit(X_treino, y_treino)
    
# Fazendo previsões
y_pred_rf = modelo_rf.predict(X_teste)
    
# Avaliando o desempenho com RMSE
RMSE = root_mean_squared_error(y_teste, y_pred_rf)
    
print(RMSE)

2.4137802205209273


#### **Gradient Boosting Regressor**

Gradient Boosting Regression é uma técnica de ensemble sequencial que constrói um modelo preditivo forte através da combinação iterativa de múltiplos modelos fracos. O termo ensemble refere-se à abordagem que combina múltiplos modelos base para criar um predictor mais robusto e preciso do que qualquer modelo individual. O algoritmo opera ajustando sequencialmente novos modelos aos resíduos (erros) dos modelos anteriores, utilizando gradiente descendente para minimizar uma função de perda diferenciável, onde cada nova árvore é treinada para prever os gradientes negativos dos erros cometidos pelas árvores existentes, criando assim um processo de melhoria contínua que gradualmente reduz o viés do modelo enquanto mantém a variância controlada através de parâmetros como taxa de aprendizado e profundidade das árvores.

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error

# Criando modelo
modelo_gbr = GradientBoostingRegressor()

# Treinando o modelo
modelo_gbr.fit(X_treino, y_treino)

# Fazendo previsões
y_pred_gbr = modelo_gbr.predict(X_teste)

# Avaliando o desempenho com RMSE
RMSE = root_mean_squared_error(y_teste, y_pred_gbr)

print(RMSE)

2.1534226663806635


#### **Elastic Net**

Elastic Net é um algoritmo de regressão regularizada que combina as penalidades L1 (Lasso) e L2 (Ridge) em uma única função objetivo. A penalidade L1 promove esparsidade nos coeficientes, efetivamente realizando seleção de variáveis ao zerar coeficientes de features menos importantes, enquanto a penalidade L2 contrai uniformemente todos os coeficientes para lidar com multicolinearidade. Durante o treinamento, o algoritmo adiciona um termo de penalização à função de custo tradicional que é uma combinação linear das normas L1 e L2 dos coeficientes, controlada pelo parâmetro alpha para a força total da regularização e l1_ratio para o balanço entre as duas penalidades, resultando em um modelo que pode efetivamente reduzir overfitting enquanto mantém a interpretabilidade através da criação de coeficientes exatamente zero para features irrelevantes.

In [10]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler

# Criando modelo
modelo_en = ElasticNet()

# Treinando o modelo
modelo_en.fit(X_treino_scaled, y_treino)

# Fazendo previsões
y_pred_en = modelo_en.predict(X_teste_scaled)

# Avaliando o desempenho com RMSE
RMSE = root_mean_squared_error(y_teste, y_pred_en)
    
print(RMSE)

2.9475110833146863


#### **Ridge Regression**

Ridge Regression é uma extensão da regressão linear ordinária que adiciona uma penalidade L2 (soma dos quadrados dos coeficientes) à função de custo para lidar com problemas de multicolinearidade e overfitting. A penalidade L2, também conhecida como regularização de Tikhonov, adiciona o quadrado da magnitude dos coeficientes à função de custo, efetivamente contraindo todos os coeficientes proporcionalmente sem eliminá-los completamente. Ao incorporar este termo de regularização controlado pelo parâmetro alpha, o algoritmo estabiliza as estimativas na presença de features altamente correlacionadas e melhora a generalização do modelo, mantendo toda a interpretabilidade da regressão linear tradicional enquanto oferece melhor performance preditiva em situações onde a matriz de design é mal-condicionada ou o número de features é grande relativo ao número de observações.

In [11]:
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.transform(X_teste)

# Criando modelo
modelo_rr = Ridge()

# Treinando o modelo
modelo_rr.fit(X_treino_scaled, y_treino)

# Fazendo previsões
y_pred_rr = modelo_rr.predict(X_teste_scaled)

# Avaliando o desempenho com RMSE
RMSE = root_mean_squared_error(y_teste, y_pred_rr)

print(RMSE)


2.037888341089658


---
### **SELEÇÃO DE ATRIBUTOS**

Dentro do dataset, há 19 features que podem ser utilizadas pelos algoritmos para tentar prever o valor do target final. Naturalmente, é de se esperar que algumas features tenham mais importância - isto é, influenciam mais no resultado final - e outras tenham menos. Nesse sentido, reduzir o número de features retirando aquelas que não possuem tanto peso no resultado final se mostra uma ótima estratégia.

Reduzir o número de atributos tem dois motivos principais que serão cruciais para o andamento do código: melhoria do desempenho computacional e prevenção de overfitting. Muitos algoritmos - como o Random Forest, de $O(n \cdot log\,n)$ - possuem complexidade que escala muito rapidamente com o número de features. Diminuir a dimensionalidade nesse contexto aumenta a viabilidade de resolução de problemas, diminuindo tempo e o uso de recursos computacionais. Além disso, com features excessivas em relação ao número de amostras, os modelos tendem a memorizar ruídos e falsos padrões nos dados de treino ao invés de aprender relações generalizáveis.

Para esse notebook, foi escolhido o atributo `feature_importance_`, usado para *ensembles* - como a Floresta Aleatória e o Gradient Boosting - e que retorna um array onde cada elemento dele é uma feature do modelo. Ele irá dizer, em pesos, o quão importante aquela feature é para o modelo. Vale ressaltar que o ideal é fazer a seleção de atributos especificamente para o algoritmo de ML que será utilizado, no entanto, optou-se por uma simplificação desse processo ao escolher apenas um processo de seleção de atributos.

O código abaixo gerou um DataFrame de saída, com a feature e sua respectiva importância, sendo esta ordenada do maior para o menor a fim de listar os 10 atributos que mais influenciam no algoritmo.

In [12]:
importancia = RandomForestRegressor(n_estimators=100, random_state=42)
importancia.fit(X_treino_norm, y_treino)
importancia_features = pd.DataFrame({
    'feature': FEATURES,
    'importance': importancia.feature_importances_
}).sort_values('importance', ascending=False)
print(importancia_features.head(10))

                     feature  importance
1                 Attendance    0.379373
0              Hours_Studied    0.244850
6            Previous_Scores    0.086903
9          Tutoring_Sessions    0.034246
3        Access_to_Resources    0.033835
2       Parental_Involvement    0.032872
14         Physical_Activity    0.027515
5                Sleep_Hours    0.024340
10             Family_Income    0.020852
16  Parental_Education_Level    0.016734


Tendo estabelecido os atributos com maior fator de importância, colocaremos-os em uma lista para que se possa fazer o split novamente, dessa vez com menos features.

In [13]:
top_10_features = importancia_features.head(10)['feature'].tolist()

X = df[top_10_features]
y = df[TARGET].values.ravel()

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

normalizador = StandardScaler()
X_treino_norm = normalizador.fit_transform(X_treino)
X_teste_norm = normalizador.transform(X_teste)

---
### **OTIMIZAÇÃO DE HIPERPARÂMETROS**

Nessa seção, busca-se estabelecer a melhor combinação de termos para potencializar o modelo. A otimização de hiperparâmetros é basicamente a criação de funções objetivo - uma expressão matemática que representa o problema a ser otimizado - especializadas para cada algoritmo, seguindo um padrão consistente de implementação.

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import optuna

Para isso, cada função específica utiliza o objeto `trial` do Optuna para definir um espaço de busca multidimensional, onde diferentes combinações de hiperparâmetros são sistematicamente exploradas e avaliadas. Cada algoritmo é colocado em uma `Pipeline`, proporcionando uma interface mais organizada para definir cada parâmetro a ser testado e avaliado pelo Optuna, que utiliza como métrica a função `cross_val_score`.

In [15]:
def objetivo_knn(trial):
    n_vizinhos = trial.suggest_int('n_neighbors', 1, 50)
    pesos = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 2)
    
    pipeline = Pipeline([
        ('regressor', KNeighborsRegressor(
            n_neighbors=n_vizinhos,
            weights=pesos,
            p=p
        ))
    ])
    
    scores_cv = cross_val_score(pipeline, X_treino, y_treino, 
                              cv=5, scoring='neg_mean_squared_error')
    return scores_cv.mean()

In [16]:
def objetivo_rf(trial):
    n_estimadores = trial.suggest_int('n_estimators', 50, 500)
    profundidade_maxima = trial.suggest_int('max_depth', 3, 20)
    min_amostras_divisao = trial.suggest_int('min_samples_split', 2, 20)
    min_amostras_folha = trial.suggest_int('min_samples_leaf', 1, 10)
    max_caracteristicas = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    
    pipeline = Pipeline([
        ('regressor', RandomForestRegressor(
            n_estimators=n_estimadores,
            max_depth=profundidade_maxima,
            min_samples_split=min_amostras_divisao,
            min_samples_leaf=min_amostras_folha,
            max_features=max_caracteristicas,
            random_state=42
        ))
    ])
    
    scores_cv = cross_val_score(pipeline, X_treino, y_treino, 
                              cv=5, scoring='neg_mean_squared_error')
    return scores_cv.mean()

In [17]:
def objetivo_gb(trial):
    n_estimadores = trial.suggest_int('n_estimators', 50, 500)
    taxa_aprendizado = trial.suggest_float('learning_rate', 0.01, 0.3)
    profundidade_maxima = trial.suggest_int('max_depth', 3, 10)
    min_amostras_divisao = trial.suggest_int('min_samples_split', 2, 20)
    min_amostras_folha = trial.suggest_int('min_samples_leaf', 1, 10)
    sub_amostra = trial.suggest_float('subsample', 0.5, 1.0)
    
    pipeline = Pipeline([
        ('regressor', GradientBoostingRegressor(
            n_estimators=n_estimadores,
            learning_rate=taxa_aprendizado,
            max_depth=profundidade_maxima,
            min_samples_split=min_amostras_divisao,
            min_samples_leaf=min_amostras_folha,
            subsample=sub_amostra,
            random_state=42
        ))
    ])
    
    scores_cv = cross_val_score(pipeline, X_treino, y_treino, 
                              cv=5, scoring='neg_mean_squared_error')
    return scores_cv.mean()

In [18]:
def objetivo_elasticnet(trial):
    alpha = trial.suggest_float('alpha', 0.001, 10.0, log=True)
    razao_l1 = trial.suggest_float('l1_ratio', 0.0, 1.0)
    
    pipeline = Pipeline([
        ('regressor', ElasticNet(
            alpha=alpha,
            l1_ratio=razao_l1,
            random_state=42,
            max_iter=10000
        ))
    ])
    
    scores_cv = cross_val_score(pipeline, X_treino, y_treino, 
                              cv=5, scoring='neg_mean_squared_error')
    return scores_cv.mean()

In [19]:
def objetivo_ridge(trial):
    alpha = trial.suggest_float('alpha', 0.001, 10.0, log=True)
    
    pipeline = Pipeline([
        ('regressor', Ridge(
            alpha=alpha,
            random_state=42,
            max_iter=10000
        ))
    ])
    
    scores_cv = cross_val_score(pipeline, X_treino, y_treino, 
                              cv=5, scoring='neg_mean_squared_error')
    return scores_cv.mean()

Após a criação das funções, deve-se iniciar estudos individuais para cada algoritmo por meio do atributo `create_study`, buscando maximizar a pontuação do modelo, a fim de buscar um modelo mais robusto. Cada estudo executa múltiplos trials, onde cada trial representa uma combinação específica de hiperparâmetros. Ao final do processo, os melhores parâmetros são identificados e serão utilizados para treinar o modelo final.

In [ ]:
print("Iniciando a otimização do modelo KNN...")
estudo_knn = optuna.create_study(direction='maximize')
estudo_knn.optimize(objetivo_knn, n_trials=100)

print("Iniciando a otimização do modelo Floresta Aleatória...")
estudo_rf = optuna.create_study(direction='maximize')
estudo_rf.optimize(objetivo_rf, n_trials=100)

print("Iniciando a otimização do modelo Gradient Boosting...")
estudo_gb = optuna.create_study(direction='maximize')
estudo_gb.optimize(objetivo_gb, n_trials=100)

print("Iniciando a otimização do modelo Elastic Net...")
estudo_en = optuna.create_study(direction='maximize')
estudo_en.optimize(objetivo_elasticnet, n_trials=100)

print("Iniciando a otimização do modelo Ridge Regression...")
estudo_ridge = optuna.create_study(direction='maximize')
estudo_ridge.optimize(objetivo_ridge, n_trials=100)

[I 2025-11-03 21:44:29,875] A new study created in memory with name: no-name-86019ecc-60c9-4b01-94ab-8e837f99628d
[I 2025-11-03 21:44:30,073] Trial 0 finished with value: -6.483037051614879 and parameters: {'n_neighbors': 38, 'weights': 'distance', 'p': 1}. Best is trial 0 with value: -6.483037051614879.


Iniciando a otimização do modelo KNN...


[I 2025-11-03 21:44:30,199] Trial 1 finished with value: -6.729812699167115 and parameters: {'n_neighbors': 47, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: -6.483037051614879.
[I 2025-11-03 21:44:30,405] Trial 2 finished with value: -6.5504865710533595 and parameters: {'n_neighbors': 48, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: -6.483037051614879.
[I 2025-11-03 21:44:30,468] Trial 3 finished with value: -11.315051372164929 and parameters: {'n_neighbors': 1, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: -6.483037051614879.
[I 2025-11-03 21:44:30,595] Trial 4 finished with value: -7.683929706896244 and parameters: {'n_neighbors': 3, 'weights': 'distance', 'p': 1}. Best is trial 0 with value: -6.483037051614879.
[I 2025-11-03 21:44:30,756] Trial 5 finished with value: -6.495116745063575 and parameters: {'n_neighbors': 21, 'weights': 'distance', 'p': 1}. Best is trial 0 with value: -6.483037051614879.
[I 2025-11-03 21:44:30,920] Trial 6 finish

Iniciando a otimização do modelo Floresta Aleatória...


[I 2025-11-03 21:44:49,066] Trial 0 finished with value: -6.102389679656065 and parameters: {'n_estimators': 386, 'max_depth': 8, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 0 with value: -6.102389679656065.
[I 2025-11-03 21:44:52,678] Trial 1 finished with value: -5.886212790636951 and parameters: {'n_estimators': 133, 'max_depth': 10, 'min_samples_split': 16, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 1 with value: -5.886212790636951.
[I 2025-11-03 21:44:57,331] Trial 2 finished with value: -5.85407460663535 and parameters: {'n_estimators': 406, 'max_depth': 16, 'min_samples_split': 15, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 2 with value: -5.85407460663535.
[I 2025-11-03 21:44:59,492] Trial 3 finished with value: -5.7875567853186 and parameters: {'n_estimators': 68, 'max_depth': 12, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 3 with value: -5.7875567853186.
[I 20

Terminado esse processo, obtém-se os melhores parâmetros com o atributo `best_params` e declara-se o trial com o melhor desempenho na otimização. Com isso, novamente cria-se pipelines de cada algoritmo, definindo o método de regressão juntamente de uma expressão cujo objetivo é iterar sobre todos os pares "chave-valor" do dicionário de melhores parâmetros; filtrá-lo, removendo qualquer chave que inicie com `regressor__`, que causa problemas posteriomente se não for retirado; e, por fim, desempacotar esse dicionário usando o `**` no início da expressão para passar os parâmetros para o construtor do regressor.

In [ ]:
melhores_params_knn = estudo_knn.best_params
melhores_params_rf = estudo_rf.best_params
melhores_params_gb = estudo_gb.best_params
melhores_params_en = estudo_en.best_params
melhores_params_ridge = estudo_ridge.best_params

pipeline_knn = Pipeline([
    ('regressor', KNeighborsRegressor(**{k: v for k, v in melhores_params_knn.items() if k != 'regressor__'}))
])

pipeline_rf = Pipeline([
    ('regressor', RandomForestRegressor(**{k: v for k, v in melhores_params_rf.items() if k != 'regressor__'}))
])

pipeline_gb = Pipeline([
    ('regressor', GradientBoostingRegressor(**{k: v for k, v in melhores_params_gb.items() if k != 'regressor__'}, random_state=42))
])

pipeline_en = Pipeline([
    ('regressor', ElasticNet(**{k: v for k, v in melhores_params_en.items() if k != 'regressor__'}, random_state=42, max_iter=10000))
])

pipeline_ridge = Pipeline([
    ('regressor', Ridge(**{k: v for k, v in melhores_params_ridge.items() if k != 'regressor__'}, random_state=42, max_iter=10000))
])

---
### **MODELOS OTIMIZADOS COM ANÁLISE DE DESEMPENHO POR VALIDAÇÃO CRUZADA**

Nesse tópico, foi feita a última verificação de desempenho dos cinco modelos escolhidos. Para tal, selecionou-se a validação cruzada, essencial para obter uma avaliação mais confiável do desempenho dos modelos. Diferente de uma simples divisão treino/teste - que por obra do acaso pode ser influenciada por uma divisão específica dos dados - a validação cruzada usa múltiplas divisões, onde cada parte dos dados serve tanto para treino quanto para teste em rodadas alternadas, os "folds".

Isso fornece uma estimativa mais robusta da performance real do modelo, permitindo identificar melhor seu verdadeiro poder de generalização para novos dados. Ademais, também revela a consistência do modelo através da variação entre as rodadas e maximiza o uso dos dados disponíveis, especialmente importante em conjuntos menores.

In [ ]:
from sklearn.model_selection import KFold

Para aplicar a validação cruzada, primeiro cria-se um dicionário que mapeia nomes às pipelines previamente configuradas com seus melhores parâmetros. Posteriormente, define-se o método de validação cruzada, que aqui é o `KFold`, e a quantidade de folds desejados, que tradicionalmente é 5 por ser um equilíbrio entre robustez e custo computacional.

Após esses processos, declara-se um loop onde para cada modelo no dicionário de pipelines, onde executa-se a validação cruzada calculando o MSE negativo que é convertido para MSE positivo e partir dele, calcula-se o RMSE. Depois é feito o treinamento, o teste e calcula-se o RMSE no conjunto de teste. Para finalizar, os resultados são armazenados e imprimidos para cada algoritmo testado, mostrando tanto a performance na validação cruzada, quanto no teste.

In [ ]:
pipelines = {
    'KNN': pipeline_knn,
    'Random Forest': pipeline_rf,
    'Gradient Boosting': pipeline_gb,
    'Elastic Net': pipeline_en,
    'Ridge Regression': pipeline_ridge
}

resultados = {}
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for nome, pipeline in pipelines.items():
    scores_cv = cross_val_score(pipeline, X_treino, y_treino, 
                              cv=kf, scoring='neg_mean_squared_error')
    scores_mse = -scores_cv
    scores_rmse = np.sqrt(scores_mse)
    
    pipeline.fit(X_treino, y_treino)
    y_previsto = pipeline.predict(X_teste)
    rmse_teste = root_mean_squared_error(y_teste, y_previsto)
    
    resultados[nome] = {
        'RMSE_CV_media': scores_rmse.mean(),
        'RMSE_CV_desvio': scores_rmse.std(),
        'RMSE_Teste': rmse_teste,
    }
    
    print(f"{nome}:")
    print(f"RMSE Validação Cruzada: {scores_rmse.mean():.4f} (±{scores_rmse.std():.4f})")
    print(f"RMSE Teste: {rmse_teste:.4f}")

---
### **EXPLICAÇÃO DOS MODELOS COM SHAP E DISCUSSÃO DE RESULTADOS**

O SHAP (SHapley Additive exPlanations) é um método de interpretabilidade baseado na teoria dos jogos que oferece explicações locais para previsões individuais e análises globais para o comportamento geral do modelo. Enquanto as explicações locais atribuem a contribuição específica de cada feature para uma única previsão, o impacto global agrega essas contribuições para todo o dataset, identificando padrões dominantes e relações estruturais. Esta capacidade de transitar entre perspectivas micro e macro torna o SHAP particularmente versátil para compreender tanto casos específicos quanto tendências gerais do modelo.

O funcionamento do SHAP baseia-se no cálculo da contribuição marginal média de cada feature, considerando todas as combinações possíveis de variáveis. Para análises locais, o método calcula exatamente como cada feature influenciou uma previsão específica. Já para o impacto global, o SHAP agrega esses valores individuais através de métricas como a média dos valores absolutos (mean |SHAP value|), criando um ranking robusto de importância de features. Esta abordagem garante que tanto a frequência quanto a magnitude do impacto de cada variável sejam consideradas, revelando interações complexas e dependências contextuais no modelo como um todo.

No presente trabalho, usaremos o SHAP para analisar o comportamento geral de cada modelo neste dataset, a fim de descobrir como cada feature contribui. Para isso, vamos começar instalando o SHAP:

In [ ]:
pip install shap

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
Em seguida, vamos ap

In [ ]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def executar_shap_todos_modelos(modelos_dict, X_treino, X_teste, feature_names, n_amostras=100):

    resultados_shap = {}
    
    # Amostrar dados para tornar SHAP mais rápido
    if len(X_teste) > n_amostras:
        indices_amostra = np.random.choice(len(X_teste), n_amostras, replace=False)
        X_teste_shap = X_teste.iloc[indices_amostra] if hasattr(X_teste, 'iloc') else X_teste[indices_amostra]
    else:
        X_teste_shap = X_teste
    
    for nome_modelo, modelo in modelos_dict.items():     
        
            # Selecionar explainer baseado no tipo de modelo
            if 'Random Forest' in nome_modelo or 'Gradient Boosting' in nome_modelo:
                explainer = shap.TreeExplainer(modelo)
                shap_values = explainer.shap_values(X_teste_shap)
                
            elif 'KNN' in nome_modelo:
                def predict_fn(X):
                    return modelo.predict(X)
                explainer = shap.KernelExplainer(predict_fn, X_treino[:100])
                shap_values = explainer.shap_values(X_teste_shap)
                
            elif 'Elastic Net' in nome_modelo or 'Ridge' in nome_modelo:
                explainer = shap.LinearExplainer(modelo, X_treino)
                shap_values = explainer.shap_values(X_teste_shap)
                
            else:
                explainer = shap.Explainer(modelo, X_treino)
                shap_values = explainer.shap_values(X_teste_shap)
            
            # Armazenar resultados
            resultados_shap[nome_modelo] = {
                'explainer': explainer,
                'shap_values': shap_values,
                'expected_value': explainer.expected_value,
                'X_teste_shap': X_teste_shap
            }
            
    
    return resultados_shap

def plot_impacto_global(resultados_shap, feature_names, modelo_nome):

    
    if resultados_shap[modelo_nome] is None:
        return
        
    dados = resultados_shap[modelo_nome]
    shap_values = dados['shap_values']
    X_teste_shap = dados['X_teste_shap']
    
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, X_teste_shap, feature_names=feature_names, 
                      plot_type="bar", show=False)
    plt.title(f'Impacto Global - {modelo_nome}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

def pipeline_shap_simplificada(modelos_dict, X_treino, X_teste, feature_names):
    
    # Calcular SHAP values para todos os modelos
    resultados_shap = executar_shap_todos_modelos(modelos_dict, X_treino, X_teste, feature_names)
    
    # Gráficos de impacto global para cada modelo
    for modelo_nome in modelos_dict.keys():
        if resultados_shap[modelo_nome] is not None:
            plot_impacto_global(resultados_shap, feature_names, modelo_nome)
    
    return resultados_shaps

modelos_dict = {
    'KNN': modelo_knn,
    'Random Forest': modelo_rf, 
    'Gradient Boosting': modelo_gbr,
    'Elastic Net': modelo_en,
    'Ridge': modelo_rr
}


resultados, importancia_modelos = pipeline_shap_simplificada(
    modelos_dict, X_treino, X_teste, FEATURES
)

NameError: name 'modelo_knn' is not defined

---
## **CONCLUSÃO**